In [1]:
import hashlib
import time
import json

class Block:
    """
    Represents a single block in the blockchain.
    Each block contains an index, timestamp, data, proof, its own hash, and the hash of the previous block to maintain linkage.
    """
    def __init__(self, index, previous_hash, timestamp, data, proof):
        self.index = index # Block number in the chain
        self.previous_hash = previous_hash #Stores the hash of the previous block in the chain
        self.timestamp = timestamp #Time the block was created
        self.data = data #Transaction or arbitrary data input
        self.proof = proof #proof of work
        self.hash = self.calculate_hash() #This block's hash value

    def calculate_hash(self):
        # Combine all key values of the block into a string
        block_string = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.proof}"
        # Encode the string and calculate the SHA-256 hash
        return hashlib.sha256(block_string.encode()).hexdigest()
    
    def to_dict(self):
        """Convert the block object into a dictionary format so it can be easily saved as JSON or displayed."""
        
        return{
            'index': self.index,
            'previous_hash': self.previous_hash,
            'timestamp': self.timestamp,
            'data': self.data,
            'proof': self.proof,
            'hash': self.hash
        }
    
    @classmethod
    def from_dict(cls,data: dict):
        """Reconstruct a Block object from a dictionary.Typically used when loading blockchain data from a JSON file."""
        return cls(
            index=data['index'],
            previous_hash=data['previous_hash'],
            timestamp=data['timestamp'],
            data=data['data'],
            proof=data['proof']
            # hash will be re-calculated automatically inside __init__                
        )




class Blockchain:
    """
    This class represents the entire blockchain system.
    It handles all the blocks and operations like adding blocks, linking them, and checking the chain.
    """
    def __init__(self):
        self.difficulty = 4  # Number of leading zeros required in a valid hash (Proof of Work)
        self.chain = [self.create_genesis_block()] #Start with first block

    def create_genesis_block(self):
        """
        This method creates the first block in the blockchain.
        It doesn't have a real previous block, so we just use "0".
        Data is set as "Genesis Block" by default
        """
        genesis_block = Block(
            index=0,
            previous_hash="0",
            timestamp=time.time(),
            data="Genesis Block",
            proof=0
        )
        return self.proof_of_work(genesis_block)

    def get_latest_block(self):
        """
        This method returns the most recent block in the blockchain.
        It's usually used when we want to add a new block and need the latest hash.
        """
        return self.chain[-1]

    def add_block(self, new_block):
        """
        This method adds a new block to the blockchain. Before appending, it links the new block to the previous one by updating its previous_hash.
        After setting the previous-hash, it recalculates the block's own hash and then adds it to the chain.
        """
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)

    def is_valid_proof(self,data,proof):
        """
        Checks if the hash of the (data + proof) starts with '0000'.
        Used during mining (proof of work).
        """
        guess = f"{data}{proof}"
        guess_hash = hashlib.sha256(guess.encode()).hexdigest()
        return guess_hash.startswith("0" * self.difficulty)
        
    def proof_of_work(self, block):
        """
        Implement the proof-of-work algorithm
        Increment the proof value until the block's hash starts with the required number of leading zeros
        """
        while not block.hash.startswith("0" * self.difficulty):
            block.proof += 1 # Keep trying different proof values
            block.hash = block.calculate_hash() #Recalculate hash with updated proof

        # when the while loop ends, we found a valid proof
        return block

    def add_data(self, data):
        """
        This method lets the user add new data to the blockchain.
        It creates a new block with that data, mines it using proof_of_work, and adds it to the chain.
        """
        # Get the latest block so we know the previous hash
        previous_block = self.get_latest_block()

        # Create a new block with updated index and data
        new_block = Block(
            index=previous_block.index + 1,
            previous_hash=previous_block.hash,
            timestamp=time.time(),
            data=data,
            proof=0
        )

        # Run proof of work to get a valid hash
        mined_block = self.proof_of_work(new_block)

        # Add the mined block to the chain
        self.add_block(mined_block)



    def is_chain_valid(self):
        """
        This method checks if the entire blockchain is still valid.
        Make sure that each block's hash is correct and properly linked to the previous one.
        """
        if len(self.chain) == 0:
            return False
        
        # Check genesis block separately
        genesis = self.chain[0]
        if (genesis.index != 0 or
            genesis.previous_hash != "0" or
            not genesis.hash.startswith("0" * self.difficulty)):
            return False
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            if (current_block.previous_hash != previous_block.hash or
                current_block.hash != current_block.calculate_hash() or
                not current_block.hash.startswith("0" * self.difficulty)):
                return False
        return True
    
    # Save and load the blockchain to a JSON file
    def save_to_file(self, filename):
         """
         Save blockchain to a file
         """
         try:
            with open(filename, 'w') as f:
                chain_data = [block.to_dict() for block in self.chain]
                json.dump(chain_data, f, indent=4)
            return True
         except Exception as e:
             print(f"Failed to save blockchain: {e} ")
             return False
    
    def load_from_file(self, filename):
        """
        Load blockchain from a file
        """
        try:
            with open(filename, 'r') as f:
                chain_data = json.load(f)
                self.chain = [Block.from_dict(data) for data in chain_data]
            return True
        except Exception as e:
            print(f"Failed to load blockchain: {e}")
            return False


# Example Usage
if __name__ == "__main__":
    blockchain = Blockchain()

    print("Mining block 1...")
    blockchain.add_data("Transaction data for Block 1")

    print("Mining block 2...")
    blockchain.add_data("Transaction data for Block 2")

    print("\nBlockchain validity:", blockchain.is_chain_valid())

    for block in blockchain.chain:
        print(f"Block {block.index} | Hash: {block.hash} | Previous Hash: {block.previous_hash}")



Mining block 1...
Mining block 2...

Blockchain validity: True
Block 0 | Hash: 0000718a1cbcac76ee471417f067c44a9c75d018590b1091f3fca30cdbb2ac0e | Previous Hash: 0
Block 1 | Hash: 0000190d6cc4498420b7768d18cff7cac565b068d59bb9322c0ac19c38033715 | Previous Hash: 0000718a1cbcac76ee471417f067c44a9c75d018590b1091f3fca30cdbb2ac0e
Block 2 | Hash: 0000e5534c00252b4264ee009fcf8253fe0b5ce5e928835501dd41b50af9984d | Previous Hash: 0000190d6cc4498420b7768d18cff7cac565b068d59bb9322c0ac19c38033715


In [27]:

def print_blockchain(chain):
    print("\n====== Blockchain Contents ======")
    for block in chain:
        print("\n-------------------------------")
        print(f"Block Index: {block.index}")
        print(f"Timestamp  : {block.timestamp}")
        print(f"Data       : {block.data}")
        print(f"Proof      : {block.proof}")
        print(f"Hash       : {block.hash}")
        print(f"Prev Hash  : {block.previous_hash}")

# Blockchain Menu
if __name__ == "__main__":
    blockchain = Blockchain()

    while True:
        print("\n==== Blockchain Menu ====")
        print("1. Add a new block")
        print("2. Display the blockchain")
        print("3. Validate the blockchain")
        print("4. Save blockchain to file")
        print("5. Load blockchain from file")
        print("6. Exit")
        
        choice = input("Enter your choice (1-6): ")

        if choice == "1":
            data = input("Enter transaction data: ")
            print("Mining block... please wait.")
            blockchain.add_data(data)
            print("Block added successfully.")
        
        elif choice == "2":
           print_blockchain(blockchain.chain)

        elif choice == "3":
            if blockchain.is_chain_valid():
                print("Blockchain is VALID.")
            else:
                print("Blockchain is INVALID!")
        
        elif choice == "4":
            filename = input("Enter filename to save blockchain: ")
            if blockchain.save_to_file(filename):
                print(f"Blockchain saved to {filename}")
            else:
                print("Failed to save blochain.")

        elif choice == "5":
            filename = input("Enter filename to load blockchain: ")
            if blockchain.load_from_file(filename):
               print(f"Blockchain loaded from {filename}")
            else:
                print("Failed to load blockchain.")
                
        elif choice == "6":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please enter 1-6.")


==== Blockchain Menu ====
1. Add a new block
2. Display the blockchain
3. Validate the blockchain
4. Save blockchain to file
5. Load blockchain from file
6. Exit
Mining block... please wait.
Block added successfully.

==== Blockchain Menu ====
1. Add a new block
2. Display the blockchain
3. Validate the blockchain
4. Save blockchain to file
5. Load blockchain from file
6. Exit
Mining block... please wait.
Block added successfully.

==== Blockchain Menu ====
1. Add a new block
2. Display the blockchain
3. Validate the blockchain
4. Save blockchain to file
5. Load blockchain from file
6. Exit

====== Blockchain Contents ======

-------------------------------
Block Index: 0
Timestamp  : 1752984522.490884
Data       : Genesis Block
Proof      : 191682
Hash       : 0000ec83bc5c8950c0e3687da08bfea6591a31fadaf98be2acf50b116e67663b
Prev Hash  : 0

-------------------------------
Block Index: 1
Timestamp  : 1752984528.175282
Data       : 1
Proof      : 284871
Hash       : 00005b0c7884721d1931